In [ ]:
from __future__ import annotations

import dataclasses
import json
import pickle
import random
import re
from pathlib import Path
from pprint import pprint

import pandas as pd
import plotly.express as px
from joblib import Parallel, delayed
from minigrid.core.constants import DIR_TO_VEC
from stable_baselines3 import PPO
from stable_baselines3.common.env_checker import check_env
from stable_baselines3.common.torch_layers import (
    FlattenExtractor,
    BaseFeaturesExtractor,
)
from tqdm.autonotebook import tqdm

import typeguard
import jaxtyping

import main as M


%load_ext autoreload
%autoreload 2

In [ ]:
print("Action space:", M.random_goal_env().action_space)
print("Observation space:", M.random_goal_env().observation_space)
check_env(M.random_goal_env())

agent_files = list(Path("agents").glob("*.zip"))
print("Number of trained agents:", len(agent_files))

## Baseline

Agent that always goes to the bottom right corner

In [ ]:
env_size = 10
M.Perfs.from_agent(
    M.BottomRightAgent(can_turn=False), episodes=100, env_size=env_size, can_turn=False
)

In [ ]:
M.show_behavior(M.BottomRightAgent(), M.random_goal_env(env_size))

# Training agents

In [ ]:
import transformer

env = M.random_goal_env(7)

print(env.__class__.__mro__)
print(env.observation_space)
transformer.CustomActorCriticPolicy(
    env.observation_space,
    env.action_space,
    lr_schedule=lambda _: 0.01,
    policy_kwargs=dict(d_model=32),
)

In [ ]:
import train
import transformer

env_size = 7
env = M.random_goal_env(env_size)

# For bottom_right_odds, None means uniform, 3 means three times more likely to be bottom right than anywhere else
for _ in range(1):
    policy, perfs = train.get_agent(
        bottom_right_prob=0.8,
        total_timesteps=100_000,
        net_arch=(20,),
        n_epochs=40,
        n_steps=8_000 // 10,
        batch_size=400,
        learning_rate=0.001,
        env_size=env_size,
        n_envs=10,
        can_turn=False,
        # policy=transformer.CustomActorCriticPolicy,
        # policy_kwargs=dict(features_extractor_class=transformer.CustomFeaturesExtractor, arch=dict(d_model=20, d_head=6, heads=3, layers=1)),
        # use_wandb=True,
        save=True,
    )

In [ ]:
M.show_behavior(policy, M.random_goal_env(env_size), 20)

In [ ]:
# Print size of the model
print("Model size:", sum(p.numel() for p in policy.policy.parameters()))

# Perfomances of agents

In [ ]:
agent_files = list(Path("agents").glob("*.zip"))
agent_files = list(Path("agents").glob("ppo_7env*.zip"))

print("Number of agents:", len(agent_files))


def get_perfs(file: Path) -> M.Perfs:
    if file.with_suffix(".json").exists():
        return M.Perfs(**json.load(open(file.with_suffix(".json"), "r")))
    else:
        env_size = int(re.search(r"(\d+)env", str(file)).group(1))
        perf = M.Perfs.from_agent(PPO.load(file), file=str(file), env_size=env_size)
        json.dump(dataclasses.asdict(perf), open(file.with_suffix(".json"), "w"))
        return perf


perfs = list(
    Parallel(n_jobs=-3)(delayed(get_perfs)(file) for file in tqdm(agent_files))
)

In [ ]:
# Scatter plot of the perfs, 2 by 2
df = pd.DataFrame(
    dict(
        br_env=[p.br_env for p in perfs],
        general_env=[p.general_env for p in perfs],
        general_br_freq=[p.general_br_freq for p in perfs],
        file=[str(p.info["file"]) for p in perfs],
        odds=[
            int(re.search(r"(\d+)odds", str(p.info["file"])).group(1)) for p in perfs
        ],
    )
)

# Plot the perfs for agent with br_env > 0.9
px.scatter(
    df[df.br_env > 0.9],
    x="general_br_freq",
    y="general_env",
    color="odds",
    hover_name="file",
    width=1000,
    height=800,
    title="Performances of agents in a 5×5 environment with random goals.",
    labels=dict(
        general_br_freq="Probability of going in the bottom right corner, regardless of where the goal is",
        general_env="Probability of reaching the goal",
        odds="Odds of goal being<br>the bottom right corner",
    ),
).show()

# Interp

In [ ]:
import torch.nn as nn

policy_net = nn.Sequential(
    policy.policy.features_extractor,
    policy.policy.mlp_extractor.policy_net,
    policy.policy.action_net,
)
value_net = nn.Sequential(
    policy.policy.features_extractor,
    policy.policy.mlp_extractor.value_net,
    policy.policy.value_net,
)

print(f"{env_size=}")
policy.policy

In [ ]:
import torch
import numpy as np

print(f"{env_size=}")
print(M.OneHotFullObsWrapper.MAPPING_NO_TURN)


def mk_obs(agent_pos: tuple[int, int], goal_pos: tuple[int, int]):
    out = torch.zeros(1, env_size - 2, env_size - 2, 4)
    out[0, agent_pos[0], agent_pos[1], 1] = 1
    out[0, goal_pos[0], goal_pos[1], 3] = 1
    return out


print("Facets: Value, Right, Down, Left, Up")
DIR_TO_VEC
for goal_pos in [(1, 1), (2, 5), (3, 4), (6, 6), (7, 7)][::-1]:
    obs = []
    for i, j in np.ndindex(env_size - 2, env_size - 2):
        obs.append(mk_obs((i, j), goal_pos))
    obs = torch.cat(obs, dim=0)

    values = value_net(obs)  # (64, 1)
    logits = policy_net(obs).softmax(-1)  # (64, 4)

    # Show heatmap of the 5 values
    heat_map = torch.cat([values, logits], dim=1)  # (64, 5)
    heat_map = heat_map.reshape(env_size - 2, env_size - 2, 5).detach().numpy()

    px.imshow(
        heat_map,
        height=500,
        title=f"Value of each position in the environment with {goal_pos=}",
        facet_col=2,
    ).show()

In [ ]:
# Show logits

obs = mk_obs((1, 1), (7, 7))
logits = policy_net(obs)  # (1, 4)
print(logits)